# Geolocation of Tree Inventory Dataset using Google Street View

This notebook outlines workflows for tree geolocation. 

The workflow combines Google Street View panoramic imagery, object detection, and depth estimation models to infer the geographic location of a tree.

Author: Thomas Lake, March 2024



# Imports

In [ ]:
# Import Libraries

# PyTorch
import torch # Version 1.13.1
print(torch.__version__)
from torchvision import transforms

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.image import imread
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# MonoDepth2
# https://github.com/nianticlabs/monodepth2
#import sys
#sys.path.insert(1, 'C:/users/talake2/Desktop/')
#sys.path.insert(1, 'C:/users/talake2/Desktop/monodepth2/')
#from monodepth2 import networks
#from monodepth2.utils import download_model_if_doesnt_exist
#from monodepth2 import layers

# ZoeDepth
# https://github.com/isl-org/ZoeDepth
import sys
sys.path.insert(1, 'C:/users/talake2/Desktop/ZoeDepth')
from zoedepth.models.builder import build_model
from zoedepth.utils.config import get_config
import timm # timm version 0.6.7 needed to load models

# Other
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import PIL.Image as pil
from PIL import UnidentifiedImageError
import os
import shutil
import numpy as np
import pandas as pd
import geopandas as gpd
from scipy.spatial import KDTree, cKDTree
import cv2
import random
import json
from math import asin, atan2, cos, degrees, radians, sin
from geopy.distance import geodesic
from shapely.geometry import Point, LineString
import pyproj
from eomaps import Maps




# Load Helper Functions

In [ ]:
##### LOAD FUNCTIONS #####

def calculate_distance(p1, p2):
    """Calculate the Euclidean distance between two (lat/lon) shapely Points in Meters."""
    return p1.distance(p2) * 111139 #multiply by 111139 to convert lat/long dist to meters (approx)

def read_image_metadata(img_folder, img_name):
    """
    Read metadata from panoramic images.
    Return metadata information: pano_id, image width, height, pano_lat, pano_lon, year, month, elevation.
    """
    img_path = os.path.join(img_folder, img_name)
    json_path = img_path.replace('.jpg', '.metadata.json')
    
    try:
        # Read metadata from panoramic image
        with open(json_path, 'r') as json_file:
            img_metadata = json.load(json_file)
    except FileNotFoundError:
        # Handle the case where metadata file is not found
        print(f"Metadata file not found for image: {img_name}")
        return None
    
    # Get metadata for panorama
    pano_id = img_metadata['panoId']
    resolution = img_metadata['resolution']
    pano_lat = img_metadata['lat']
    pano_lon = img_metadata['lng']
    date = img_metadata['date']
    elevation = img_metadata['elevation']
    
    return pano_id, resolution, pano_lat, pano_lon, date, elevation
    

def read_panoramic_image_and_metadata(img_folder, img_name):
    """
    Read panoramic image and its metadata from the specified folder.
    Returns the image, height, width, channels, pano_id, pano_rotation_value, pano_lat, pano_lon.
    """
    img_path = os.path.join(img_folder, img_name)
    json_path = img_path.replace('.jpg', '.metadata.json')
    
    try:
        # Read panoramic image
        img = imread(img_path)
        height, width, channels = img.shape
        # Check if image size is correct. Images should be shape (8192,16384,3).
        if width != 16384:
            print("Resizing GSV Image to Dimensions (16384, 8192)")
            img = cv2.resize(img, dsize = (16384, 8192), interpolation = cv2.INTER_AREA)
        height, width, channels = img.shape
    except (FileNotFoundError, UnidentifiedImageError):
        # Handle the case where image file is not found or is not a valid image
        print(f"Image file not found or is not a valid image: {img_name}")
        return None
    
    try:
        # Read panoramic image metadata
        with open(json_path, 'r') as json_file:
            img_metadata = json.load(json_file)
    except FileNotFoundError:
        # Handle the case where metadata file is not found
        print(f"Metadata file not found for image: {img_name}")
        return None

    # Get metadata for panorama
    pano_id = img_metadata['panoId']
    pano_rotation_value = img_metadata['rotation']  # North
    pano_lat = img_metadata['lat']
    pano_lon = img_metadata['lng']

    return img, width, pano_id, pano_rotation_value, pano_lat, pano_lon


def calculate_pano_tree_angle(row, img_width, pano_rotation_north):
    """
    Calculate the angle of rotation of a tree in the panoramic image, based on the 
    vertical center of the bounding box given from a YOLO model.
    """
    # Extract vertical center of the tree bounding box
    tree_center_x = (row['xmin'] + row['xmax']) / 2
    # Calculate the angle relative to the north
    px_per_degree = (img_width/360) # eq. 45.5111
    tree_angle = tree_center_x / px_per_degree
    # Adjust the angle based on the north rotation value
    adjusted_angle = (tree_angle - pano_rotation_north) % 360
    return adjusted_angle

def get_point_at_distance(lat1, lon1, d, bearing, R=6371):
    """
    lat: initial latitude, in degrees
    lon: initial longitude, in degrees
    d: target distance from initial
    bearing: (true) heading in degrees
    R: optional radius of sphere, defaults to mean radius of earth

    Returns new lat/lon coordinate {d}km from initial, in degrees
    Source: https://stackoverflow.com/questions/7222382/get-lat-long-given-current-point-distance-and-bearing
    """
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    a = radians(bearing)
    d = d/1000 # Convert from Km to meters
    lat2 = asin(sin(lat1) * cos(d/R) + cos(lat1) * sin(d/R) * cos(a))
    lon2 = lon1 + atan2(
        sin(a) * sin(d/R) * cos(lat1),
        cos(d/R) - sin(lat1) * sin(lat2)
    )
    return (degrees(lat2), degrees(lon2),)

def extend_lines(row, distance):
    '''
    Create a shapely.LineString (line) between a panoramic image and detected tree
    Given the panoramic image origin (lat, long), and the bearing (angle of tree),
    create a line with a distance.
    '''
    extended_point = get_point_at_distance(row['Pano_Origin_Lat'], row['Pano_Origin_Lon'], distance, row['Pano_Tree_Angle'])
    return LineString([(row['Pano_Origin_Lon'], row['Pano_Origin_Lat']), (extended_point[1], extended_point[0])])


def process_yolo_results(yolo_results_df, num_detections):
    '''
    Process results of a YOLO model that detects trees in a panoramic image.
    For geolocation: select trees with large bounding boxes that are nearer to the image orign
    Calculate area and aspect ratio of each detected tree by bounding box coordinates
    Filter by the largest trees with high detection confidence and small aspect ratio
    
    Parameter: num_detections. Integer specifying how many detected trees are selected by bounding box area (N-largest trees).
    '''
    # Check if the dataframe is empty
    if model_results_df.empty:
        print("No trees detected in the image.")
        return None

    # Calculate bounding box area and aspect ratio
    model_results_df['area'] = (model_results_df['xmax'] - model_results_df['xmin']) * (model_results_df['ymax'] - model_results_df['ymin'])
    model_results_df['aspect_ratio'] = (model_results_df['xmax'] - model_results_df['xmin']) / (model_results_df['ymax'] - model_results_df['ymin'])

    # Set the display format for the area column to non-scientific
    pd.set_option('display.float_format', lambda x: '%.3f' % x)
    
    # Select detection by bounding box area. Specify how many detected objects to return with num_detections.
    topN_by_area = model_results_df.nlargest(num_detections, 'area', 'all')

    # Exclude values with aspect ratio above 1.5
    topN_filtered = topN_by_area[(topN_by_area['aspect_ratio'] <= 2.0) & (topN_by_area['confidence'] > 0.6)]

    return topN_filtered

def predict_genus_tencrop_cnn(img, model, xmin, ymin, xmax, ymax):
    '''
    Function to apply a trained CNN model to classify trees detected in street view imagery.
    Classifies with TenCrop: https://pytorch.org/vision/0.15/generated/torchvision.transforms.TenCrop.html#torchvision.transforms.TenCrop
    Takes in a panoramic street view image, applies transformations (crops, normalize) and runs prediction.
    Returns the most likely class prediction (argmax) and a dictionary of the predicted class probabilities.
    '''
    # Genera to detect in google street view images
    selected_genera = ['acer', 'ailanthus', 'betula', 'citrus', 'cupaniopsis', 'erythrina', 'fraxinus', 'gleditsia',
                       'juglans', 'juniperus', 'magnolia', 'phoenix', 'picea', 'pinus', 'prunus', 'pseudotsuga',
                       'pyrus', 'quercus', 'rhus', 'sequoia', 'taxodium', 'thuja', 'tilia', 'ulmus', 'washingtonia']

    # Crop image to detected tree bounding box
    crop_img = img[ymin:ymax, xmin:xmax, ...]

    # Check if image is at least 512 x 512 to run predictions
    # Create larger bounding box around detected tree if dimensions are too small
    if crop_img.shape[0] < 512:
        print("Re-cropping image to meet required dimensions: 512 x 512")
        ymin = max(0, ymin - (512 - crop_img.shape[0]) // 2)
        ymax = ymin + 512
        crop_img = img[ymin:ymax, xmin:xmax, ...]

    if crop_img.shape[1] < 512:
        print("Re-cropping image to meet required dimensions: 512 x 512")
        xmin = max(0, xmin - (512 - crop_img.shape[1]) // 2)
        xmax = xmin + 512
        crop_img = img[ymin:ymax, xmin:xmax, ...]    

        
    # Define transformations
    transform = transforms.Compose([
        transforms.ToPILImage(),  # Convert to PIL Image
        transforms.TenCrop(512),  # Apply TenCrop augmentation
        transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),  # Convert cropped images to tensor
        transforms.Lambda(lambda crops: torch.stack([transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))(crop) for crop in crops]))  # Normalize each cropped image
    ])

    # Apply transformations
    cropped_imgs = transform(crop_img)

    # Run CNN model prediction on each cropped image
    predictions = []
    for cropped_img_tensor in cropped_imgs:
        cropped_img_tensor = cropped_img_tensor.unsqueeze(0).cuda()  # add first dimension of batch size and push to GPU
        output = model(cropped_img_tensor)
        predictions.append(output)

    # Aggregate predictions from all crops
    aggregated_output = torch.mean(torch.stack(predictions), dim=0)

    # Get the softmax class probability for each genus
    class_probs = torch.softmax(aggregated_output, dim=1)

    # Get the argmax predicted class
    class_argmax = torch.argmax(aggregated_output).item()

    # From predicted class index, get genus name
    predicted_tree_genus = selected_genera[class_argmax]

    # Initialise a dictionary to hold the output genus probabilities
    genus_softmax_dict = {}
    for i, genus in enumerate(selected_genera):
        genus_softmax_dict[f'{genus}_prob'] = class_probs[0][i].item()

    return predicted_tree_genus, genus_softmax_dict

# Load Object Detection and Depth Estimation Models

In [ ]:
# Models: Object Detection and Depth Estimation

### YOLO Model to Detect Trees ###
# Load a pretrained YOLO model that detects trees
model1_name = 'tree-detector-yolov5x-oct2323-autoarborist-25epochs'
model1_path = f'C:/Users/talake2/Desktop/auto_arborist_cvpr2022_v015/yolov5/runs/train/{model1_name}/weights/last.pt'
tree_model = torch.hub.load(r'C:/Users/talake2/Desktop/auto_arborist_cvpr2022_v015/yolov5', 'custom', path=model1_path, source='local')
tree_model.conf = 0.25  # confidence threshold (0-1)

# Paths for MonoDepth2 Encoder and Decoder
# https://github.com/nianticlabs/monodepth2
encoder_path = "C:/users/talake2/Desktop/monodepth2/models/mono+stereo_1024x320/encoder.pth"
depth_decoder_path = "C:/users/talake2/Desktop/monodepth2/models/mono+stereo_1024x320/depth.pth"

# Load ZoeDepth Model
# ZoeD_K
conf = get_config("zoedepth", "infer", config_version="kitti", config_mode="eval")
model_zoe_k = build_model(conf)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
zoe = model_zoe_k.to(DEVICE)

# Load Tree Inventory Data

In [ ]:
# Geolocation Inputs: Tree Inventories and Panoramic Image

### Tree Inventory ###
# Read in Tree Inventory .csv file containing 'latitude' and 'longitude' columns information on tree locations
tree_inventory_df = pd.read_csv(r'C:/Users/talake2/Desktop/auto_arborist_cvpr2022_v015/auto_arborist_tfrecords_data/auto_arborist_tfrecords_csv/tfrecords_locs_all.csv')
# Create a GeoDataFrame from the tree_inventory_df
tree_geometry = [Point(lon, lat) for lon, lat in zip(tree_inventory_df['Longitude'], tree_inventory_df['Latitude'])]
tree_gdf = gpd.GeoDataFrame(tree_inventory_df, geometry=tree_geometry)


### Google Street View Panoramic Images ###
# Define directory containing panoramic images and .JSON metadata files
testing_city = r'sioux-falls'
img_folder = f'C:/Users/talake2/Desktop/tree-geolocation/geolocation-panos-testing-tiny/{testing_city}'



# Load Image Classifier (CNN)

In [ ]:
# Prepare a Basic Model for Image Classification

import torch.nn as nn # contains base class for all neural network modules
import torch.nn.functional as F #https://pytorch.org/docs/stable/nn.functional.html contains common functions for training NNs (convolutions, losses, etc..)
import torchvision # version 0.16.2

class ImageClassificationBase(nn.Module): # https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module
    # Define a base class with functionality for model training, validation, and evaluation per epoch
    
    def training_step(self, batch):
        images, labels = batch
        out = self(images) # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch
        out = self(images) # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        acc = accuracy(out, labels) # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


In [ ]:
# Define a CNN Model using ResNet50
class ResNet50ImageClassification(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Load the pre-trained ResNet model
        self.network = torchvision.models.resnet50(weights='ResNet50_Weights.IMAGENET1K_V2')
        # Modify the final fully connected layer to match the number of classes in your dataset
        num_classes = 25 # Set number of output classes
        in_features = self.network.fc.in_features
        self.network.fc = nn.Linear(in_features, num_classes)

    def forward(self, xb):
        return self.network(xb)
    
    
# # Define a CNN Model using EfficientNetV2-S
class EfficientNetImageClassification(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Load the pre-trained EfficientNetV2-L Model
        self.network = torchvision.models.efficientnet_v2_s(pretrained=True)
        # Modify the final fully connected layer to match the number of classes in your dataset
        num_classes = 25 # Set number of output classes
        in_features = self.network.classifier[1].in_features
        self.network.classifier = nn.Linear(in_features, num_classes)

    def forward(self, xb):
        return self.network(xb)


In [ ]:
# Helper function and class to load data to GPU

def get_default_device():
    """ Set Device to GPU or CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    

def to_device(data, device):
    "Move data to the device"
    if isinstance(data,(list,tuple)):
        return [to_device(x,device) for x in data]
    return data.to(device,non_blocking = True)

class DeviceDataLoader():
    """ Wrap a dataloader to move data to a device """
    
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    
    def __iter__(self):
        """ Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b,self.device)
            
    def __len__(self):
        """ Number of batches """
        return len(self.dl)
    

In [ ]:
# Load a pre-trained model
model_path = r'C:\Users\talake2\Desktop\auto_arborist_cvpr2022_v015\pytorch_cnn_classifier\pytorch_cnn_autoarborist_inaturalist_models_march2024\tree-classification-autoarb-25-genera-1000imgs-effnet2s-10epochs-lr001-march724\last.pth'

# Get GPU Device
device = get_default_device()
device

# Instantiate the model with the same architecture as the model which parameters you saved
model = EfficientNetImageClassification()

#load the model to the device
model = to_device(EfficientNetImageClassification(), device)

model.load_state_dict(torch.load(model_path))

model.eval() # Call model.eval() to set dropout and batch normalization layers to evaluation mode before running inference. Failing to do this will yield inconsistent inference results.


# Apply classification on single image

In [ ]:
# Apply classification on single image
# Constant for classes    
selected_genera = ['acer','ailanthus','betula','citrus','cupaniopsis','erythrina','fraxinus','gleditsia','juglans','juniperus',
                   'magnolia','phoenix','picea','pinus','prunus','pseudotsuga','pyrus','quercus','rhus','sequoia','taxodium',
                   'thuja','tilia','ulmus','washingtonia']

img_path = r'C:/Users/talake2/Desktop/auto_arborist_cvpr2022_v015/auto_arborist_jpegs/jpegs_aerial_streetlevel_raw/all_cities_streetview/train/acer/streetlevel_4_7.jpg'
img = imread(img_path)
img.shape

# Resize the image
img = cv2.resize(img, (512, 512))

# Convert image to tensor using torchvision.transforms.ToTensor()
transform = transforms.Compose([transforms.ToTensor()])

img = transform(img) #Transforms image to [0-1] and channels first
img = img.unsqueeze(0) #add first dimension of batch size
img = img.cuda() #push to GPU

# Predict on an image
output = model(img)

# Get the index of the maximum value in the output tensor
predicted_class_index = torch.argmax(output)

# Use the index to get the corresponding class label
predicted_class_label = selected_genera[predicted_class_index.item()]

print("Predicted Class Label:", predicted_class_label)

# Get the top 5 class predictions
top5_probabilities, top5_indices = torch.topk(output, 5)

# Convert indices to class labels
top5_class_labels = [selected_genera[i] for i in top5_indices.squeeze().tolist()]

print("Top 5 Predicted Class Labels:", top5_class_labels)
print("Top 5 Predicted Probabilities:", top5_probabilities)

# Apply classification with TenCrop

In [ ]:
# Genera to detect in google street view images
selected_genera = ['acer', 'ailanthus', 'betula', 'citrus', 'cupaniopsis', 'erythrina', 'fraxinus', 'gleditsia',
                   'juglans', 'juniperus', 'magnolia', 'phoenix', 'picea', 'pinus', 'prunus', 'pseudotsuga',
                   'pyrus', 'quercus', 'rhus', 'sequoia', 'taxodium', 'thuja', 'tilia', 'ulmus', 'washingtonia']

# Define transformations
transform = transforms.Compose([
    transforms.ToPILImage(),  # Convert to PIL Image
    transforms.TenCrop(512),  # Apply TenCrop augmentation
    transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),  # Convert cropped images to tensor
    transforms.Lambda(lambda crops: torch.stack([transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))(crop) for crop in crops]))  # Normalize each cropped image
])

img_path = r'C:/Users/talake2/Desktop/auto_arborist_cvpr2022_v015/auto_arborist_jpegs/jpegs_aerial_streetlevel_raw/all_cities_streetview/train/acer/streetlevel_4_7.jpg'
img = imread(img_path)
img.shape

# Apply transformations: ten cropping, normalization
cropped_imgs = transform(img)

# Run CNN model prediction on each cropped image
predictions = [] # A Tensor of 10 predictions, each with 25 class probabilities
for cropped_img_tensor in cropped_imgs:
    cropped_img_tensor = cropped_img_tensor.unsqueeze(0).cuda()  # add first dimension of batch size and push to GPU
    output = model(cropped_img_tensor) # CNN prediction on image
    predictions.append(output)

# Aggregate/ average across 10 predictions from all crops
aggregated_output = torch.mean(torch.stack(predictions), dim=0)

# Get the softmax class probability for each genus
class_probs = torch.softmax(aggregated_output, dim=1)

# Get the argmax predicted class genus
class_argmax = torch.argmax(aggregated_output).item()

# From predicted class index, get genus name
predicted_tree_genus = selected_genera[class_argmax]

# Initialise a dictionary to hold the output genus probabilities
genus_softmax_dict = {}
for i, genus in enumerate(selected_genera):
    genus_softmax_dict[f'{genus}_prob'] = class_probs[0][i].item()

In [ ]:
predictions

# Plotting Panoramic Image Locations and Tree Inventory Data on Satellite Map

In [ ]:
# Plot Pano Locations, Estimated Tree Locations, Lines and Intersections
from eomaps import Maps
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io import img_tiles
%matplotlib inline

panos_df = pd.DataFrame(columns=['Pano_ID', 'Pano_Origin_Lon', 'Pano_Origin_Lat'])

# Iterate over each image in the directory.
for img_name in os.listdir(img_folder):
    if img_name.endswith('.jpg'):
        
        # Read Panoramic image and Metadata .JSON file, return image and metadata
        pano_id, resolution, pano_lat, pano_lon, date, elevation = read_image_metadata(img_folder, img_name)

        new_row = {
        'Pano_ID': pano_id,
        'Pano_Origin_Lon': pano_lon,
        'Pano_Origin_Lat': pano_lat,
        }
                        
        panos_df = pd.concat([panos_df, pd.DataFrame([new_row])], ignore_index=True)
        
# Define the bounding box for the plotting area
bbox = [
    panos_df['Pano_Origin_Lon'].min() - 0.0003,
    panos_df['Pano_Origin_Lon'].max() + 0.0003,
    panos_df['Pano_Origin_Lat'].min() - 0.0003,
    panos_df['Pano_Origin_Lat'].max() + 0.0003
]

# Crop the tree invenntory records to the bounding box where trees are predicted
cropped_tree_inventory = tree_inventory_df[
    (tree_inventory_df['Longitude'] >= bbox[0]) &
    (tree_inventory_df['Longitude'] <= bbox[1]) &
    (tree_inventory_df['Latitude'] >= bbox[2]) &
    (tree_inventory_df['Latitude'] <= bbox[3])
]

# Convert panos_df to a GeoDataFrame
geometry = [Point(xy) for xy in zip(panos_df['Pano_Origin_Lon'], panos_df['Pano_Origin_Lat'])]
panos_gdf = gpd.GeoDataFrame(panos_df, geometry=geometry, crs="EPSG:4326")

# Reproject pano gdf to meters
panos_gdf_reproj = panos_gdf.to_crs(epsg=3395) # Reproject to UTM Zome 13N Meters 32613
#panos_gdf_reproj.plot()

# Create a 20 meter buffer around the tree_inventory_gdf points
buffer_distance = 20  # in meters
panos_gdf_reproj_buffer = panos_gdf_reproj.geometry.buffer(buffer_distance)
panos_gdf_reproj_buffer_polygon = panos_gdf_reproj_buffer.geometry.unary_union
#panos_gdf_reproj_buffer_polygon

# Convert cropped_tree_inventory to a Geodataframe
tree_geometry = [Point(xy) for xy in zip(cropped_tree_inventory['Longitude'], cropped_tree_inventory['Latitude'])]
tree_inventory_gdf = gpd.GeoDataFrame(cropped_tree_inventory, geometry=tree_geometry, crs="EPSG:4326")

tree_inv_gdf_reproj = tree_inventory_gdf.to_crs(epsg=3395) # Reproject to UTM Zome 13N Meters
#tree_inv_gdf_reproj.plot()

# Subset tree inventory to within buffer distance of panoramic images
tree_inv_within_panos = tree_inv_gdf_reproj[tree_inv_gdf_reproj.geometry.within(panos_gdf_reproj_buffer_polygon)]

# Convert the tree_inventory_gdf to EPSG4326
tree_inventory_gdf = tree_inv_within_panos.to_crs(epsg=4326)
# Reset index
tree_inventory_gdf.reset_index(drop=True, inplace=True)

# Plotting:
# Define a map and extent
m = Maps(crs=Maps.CRS.Mercator.GOOGLE, figsize=(20, 20))
m.set_extent((bbox[0], bbox[1], bbox[2], bbox[3]))

# Plot the GeoDataFrame onto the map
m.add_gdf(panos_gdf, marker='x', color='blue', alpha=0.80, markersize = 250, label='Panoramic Points')
m.add_gdf(tree_inventory_gdf, marker='^', color='orange', alpha=0.80, markersize = 250, label='Tree Inventory')
m.add_wms.ESRI_ArcGIS.SERVICES.World_Imagery.add_layer.xyz_layer()

# Show the map
#m.savefig(f"C:/Users/talake2/Desktop/tree-geolocation/geolocation-ailanthus-testing-cities-results/{testing_city}/{testing_city}-geolocation-ailanthus-map.png")
m.show();


# Run Initial Tree Geolocation

In [ ]:

# Store resulting data from tree detection and initial geolocation
tree_geolocation_results_df = pd.DataFrame(columns=['Pano_ID', 'Pano_Origin_Lon', 'Pano_Origin_Lat', 'Est_Tree_Lon', 'Est_Tree_Lat', 'Predicted_Genus', 'Pano_Tree_Angle', 'Est_Depth', 'Bbox_Area', 'Bbox_Aspect'])

# Multiply depth estimates by 3.0 to correct for scale distortion in equirectangular panoramic images
DEPTH_SCALING_FACTOR = 3.0 # Default scaling factor, seems to work decent for close-trees but not for far-trees

# Iterate over each image in the directory.
for img_name in os.listdir(img_folder):
    if img_name.endswith('.jpg'):
        
        # Read Panoramic image and Metadata .JSON file, return image and metadata
        img, width, pano_id, pano_rotation_value, pano_lat, pano_lon = read_panoramic_image_and_metadata(img_folder, img_name)

        print(f"Running Tree Detection and Initial Geolocation on: {pano_id}")

        # Apply YOLO model to detect all trees in image
        model_results = tree_model(img) # Run YOLO Inference
        model_results_df = model_results.pandas().xyxy[0] # Store YOLO results

        # Subset trees from YOLO model by detection heuristics: bounding box size, aspect ratio, confidence
        top_detection_results = process_yolo_results(model_results_df, 3)
        print(top_detection_results)

        if top_detection_results is not None: # if trees are detected in the image:
            
            # Depth Estimation with ZoeDepth
            metric_depth_resized_zoe = zoe.infer_pil(img)

            # Geolocate all detected trees
            for index, row in top_detection_results.iterrows():

                # Get bounding box for detected tree in the image
                xmin, ymin, xmax, ymax = (
                    int(row['xmin']),
                    int(row['ymin']),
                    int(row['xmax']),
                    int(row['ymax'])
                )
                
                # Run Ten-Crop CNN Model Classification on Detected Tree
                predicted_genus, genus_softmax_dict = predict_genus_tencrop_cnn(img, model, xmin, ymin, xmax, ymax)

                # Crop the depth map to the tree matched from panoramic image to inventory
                cropped_depth = metric_depth_resized_zoe[ymin:ymax, xmin:xmax]

                # Estimate depth as the median of the depthmap
                est_tree_depth = np.median(cropped_depth) * DEPTH_SCALING_FACTOR

                # Get index and rotation (0-360 degrees) for detected tree in the panoramic image
                tree_rotation_value = calculate_pano_tree_angle(row, width, pano_rotation_value)
                
                # Geolocate tree given pano origin, distance, and bearing
                est_tree_lat, est_tree_lon = get_point_at_distance(pano_lat, pano_lon, est_tree_depth, tree_rotation_value)

                # Capture outputs for each predicted tree as a new_row in a dataframe
                new_row = {
                    'Pano_ID': pano_id,
                    'Pano_Origin_Lon': pano_lon,
                    'Pano_Origin_Lat': pano_lat,
                    'Est_Tree_Lon': est_tree_lon,
                    'Est_Tree_Lat': est_tree_lat,
                    'Predicted_Genus': predicted_genus,
                    'Pano_Tree_Angle': tree_rotation_value,
                    'Est_Depth': est_tree_depth,
                    'Bbox_Area': row['area'],
                    'Bbox_Aspect': row['aspect_ratio']
                }

                # Extend the new_row dictionary with predicted genus softmax values
                new_row.update(genus_softmax_dict)

                # Concatenate results to output dataframe
                tree_geolocation_results_df = pd.concat([tree_geolocation_results_df, pd.DataFrame([new_row])], ignore_index=True)

        else: print("No trees detected in the image.")



# Initial Tree Geolocation Estimates

In [ ]:
pd.set_option('display.max_columns', None)
tree_geolocation_results_df

In [ ]:
# Export initial geolocation estimates as .csv
tree_geolocation_results_df.to_csv(f"C:/Users/talake2/Desktop/tree-geolocation/geolocation-panos-testing-tiny/{testing_city}-initial-tree-detection-geolocation-results-argmax-genus.csv")


# Plot Initial Geolocation Estimates

In [ ]:
# Plot Pano Locations, Estimated Tree Locations, Lines and Intersections
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io import img_tiles
%matplotlib inline

# Define the bounding box for the plotting area
bbox = [
    tree_geolocation_results_df['Est_Tree_Lon'].min() - 0.0003,
    tree_geolocation_results_df['Est_Tree_Lon'].max() + 0.0003,
    tree_geolocation_results_df['Est_Tree_Lat'].min() - 0.0003,
    tree_geolocation_results_df['Est_Tree_Lat'].max() + 0.0003
]

# Crop the tree invenntory records to the bounding box where trees are predicted
cropped_tree_inventory = tree_inventory_df[
    (tree_inventory_df['Longitude'] >= bbox[0]) &
    (tree_inventory_df['Longitude'] <= bbox[1]) &
    (tree_inventory_df['Latitude'] >= bbox[2]) &
    (tree_inventory_df['Latitude'] <= bbox[3])
]


# Convert panos_df to a GeoDataFrame
geometry = [Point(xy) for xy in zip(tree_geolocation_results_df['Pano_Origin_Lon'], tree_geolocation_results_df['Pano_Origin_Lat'])]
panos_gdf = gpd.GeoDataFrame(tree_geolocation_results_df, geometry=geometry, crs="EPSG:4326")

# Convert cropped_tree_inventory to a Geodataframe
tree_geometry = [Point(xy) for xy in zip(cropped_tree_inventory['Longitude'], cropped_tree_inventory['Latitude'])]
tree_inventory_gdf = gpd.GeoDataFrame(cropped_tree_inventory, geometry=tree_geometry, crs="EPSG:4326")


# Define unique colors for each predicted genus
unique_genera = tree_geolocation_results_df['Predicted_Genus'].unique()
num_colors = len(unique_genera)
color_map = plt.cm.get_cmap('tab10', num_colors)  # You can choose any colormap here

# Create a dictionary mapping predicted genera to colors
genus_color_dict = {genus: color_map(i) for i, genus in enumerate(unique_genera)}

# Convert estimated tree geolocations to a Geodataframe - categorize by detected class (here: trees and ailanthus)
tree_est_geometry = [Point(xy) for xy in zip(tree_geolocation_results_df['Est_Tree_Lon'], tree_geolocation_results_df['Est_Tree_Lat'])]
tree_est_gdf = gpd.GeoDataFrame(tree_geolocation_results_df, geometry=tree_est_geometry, crs="EPSG:4326")
# Apply colors based on predicted genus
tree_est_gdf['color'] = tree_est_gdf['Predicted_Genus'].map(genus_color_dict)


# Define a map and extent
m = Maps(crs=Maps.CRS.Mercator.GOOGLE, figsize=(20, 20))
m.set_extent((bbox[0], bbox[1], bbox[2], bbox[3]))

# Plot the GeoDataFrame onto the map
m.add_gdf(panos_gdf, marker='x', color='blue', alpha=0.75, markersize = 300, label='Panoramic Points')
m.add_gdf(tree_inventory_gdf, marker='^', color='orange', alpha=0.75, markersize = 300, label='Tree Inventory')
m.add_gdf(tree_est_gdf, marker='.', color=tree_est_gdf['color'], alpha=1.0, markersize=600, label='Estimated Tree Geolocation')


m.add_wms.ESRI_ArcGIS.SERVICES.World_Imagery.add_layer.xyz_layer()

# Show and save the map
#m.savefig(f"C:/Users/talake2/Desktop/tree-geolocation/geolocation-panos-testing-cities-results/{testing_city}-initial-tree-geolocation-estimates.png")
m.show();


# Geolocate Trees with Triangulation

In [ ]:
# Plot lines from panoramic image origin to estimated tree location

In [ ]:
# Create Lines from Panoramic Image to Detected Tree

# Distance of line from origin to detected tree (meters)
line_distance = 20

# Group the tree_geolocation_results_df by Pano_ID and apply LineString to each group
panos_grouped = tree_geolocation_results_df.groupby('Pano_ID').apply(lambda x:[extend_lines(row, line_distance) for idx, row in x.iterrows()])

# Each line connects the panoramic image origin to the estimated tree location
lines = []
for sublist in panos_grouped:
    for line in sublist:
        lines.append(line)
        
print(f'Created', len(lines), "lines between panoramic image and detected trees")

# Create a GeoDataFrame from the lines
lines_gdf = gpd.GeoDataFrame(geometry=lines, columns=['geometry'], crs="EPSG:4326")

# Combine the detected and classified trees with lines
combined_geolocation_lines = pd.concat([tree_geolocation_results_df, lines_gdf], axis=1)
combined_geolocation_lines

# Triangulate lines to create intersections 

In [ ]:

# Create an empty DataFrame to store outputs
triangulated_tree_detections = pd.DataFrame(columns=['Intersection', 'Pano_ID_1', 'Pano_Origin_Lon_1', 'Pano_Origin_Lat_1', 
                                   'Est_Depth_1', 'Pano_ID_2', 'Pano_Origin_Lon_2', 
                                   'Pano_Origin_Lat_2', 'Est_Depth_2'])

tolerance = 1e-6  # Tolerance for overlapping intersections based on location coordinates

# Iterate through all detected trees and triangulate detected trees
for i in range(len(combined_geolocation_lines)):
    # Get the origin of the current line
    origin = combined_geolocation_lines.geometry[i].coords[0]
    for j in range(i+1, len(combined_geolocation_lines)):
        if combined_geolocation_lines.geometry[i].intersects(combined_geolocation_lines.geometry[j]): # If the two lines intersect
            intersection_point = combined_geolocation_lines.geometry[i].intersection(combined_geolocation_lines.geometry[j]) # Get the intersection point
            # Check if intersection is not the origin of the current line and the other line
            if (abs(intersection_point.x - origin[0]) > tolerance or abs(intersection_point.y - origin[1]) > tolerance) and \
               (abs(intersection_point.x - lines[j].coords[0][0]) > tolerance or abs(intersection_point.y - lines[j].coords[0][1]) > tolerance):
                           
                # Get Pano ID and Location For Intersection Point 1
                pano_i_id = combined_geolocation_lines.iloc[i]['Pano_ID']
                pano_i_lon = combined_geolocation_lines.iloc[i]['Pano_Origin_Lon']
                pano_i_lat = combined_geolocation_lines.iloc[i]['Pano_Origin_Lat']
                est_depth_i = combined_geolocation_lines.iloc[i]['Est_Depth']
                
                # Get Pano ID and Location For Intersection Point 2
                pano_j_id = combined_geolocation_lines.iloc[j]['Pano_ID']
                pano_j_lon = combined_geolocation_lines.iloc[j]['Pano_Origin_Lon']
                pano_j_lat = combined_geolocation_lines.iloc[j]['Pano_Origin_Lat']
                est_depth_j = combined_geolocation_lines.iloc[j]['Est_Depth']
                
                # Get the class probabilities for image i and j
                pano_i_genus_probs = combined_geolocation_lines.iloc[i, 10:35] # Probabilities for 25 classes of tree genera prediction from CNN model
                pano_j_genus_probs = combined_geolocation_lines.iloc[j, 10:35] # Probabilities for 25 classes of tree genera prediction from CNN model
                
                # Combine and average the class probabilities
                combined_probs = pano_i_genus_probs.add(pano_j_genus_probs)
                average_genus_probs = combined_probs / 2
                
                # Append the data to the output DataFrame
                output_data = {
                    'Intersection': intersection_point,
                    'Pano_ID_1': pano_i_id, 
                    'Pano_Origin_Lon_1': pano_i_lon, 
                    'Pano_Origin_Lat_1': pano_i_lat, 
                    'Est_Depth_1': est_depth_i,
                    'Pano_ID_2': pano_j_id, 
                    'Pano_Origin_Lon_2': pano_j_lon, 
                    'Pano_Origin_Lat_2': pano_j_lat, 
                    'Est_Depth_2': est_depth_j,
                    **average_genus_probs.to_dict()  # Convert average_genus_probs to a dictionary and unpack into the DataFrame
                }
                triangulated_tree_detections = pd.concat([triangulated_tree_detections, pd.DataFrame([output_data])], ignore_index=True)

                
# Convert to a GeoDataFrame
intersection_geometry = [Point(xy) for xy in triangulated_tree_detections['Intersection']]
triangulated_tree_detections_gdf = gpd.GeoDataFrame(triangulated_tree_detections, geometry=intersection_geometry, crs="EPSG:4326")

triangulated_tree_detections_gdf

# Plot Triangulated Geolocation Estimates

In [ ]:
# Plot Pano Locations, Estimated Tree Locations, Lines and Intersections
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io import img_tiles
%matplotlib inline

# Define the bounding box for the plotting area
bbox = [
    tree_geolocation_results_df['Est_Tree_Lon'].min() - 0.0003,
    tree_geolocation_results_df['Est_Tree_Lon'].max() + 0.0003,
    tree_geolocation_results_df['Est_Tree_Lat'].min() - 0.0003,
    tree_geolocation_results_df['Est_Tree_Lat'].max() + 0.0003
]

# Crop the tree invenntory records to the bounding box where trees are predicted
cropped_tree_inventory = tree_inventory_df[
    (tree_inventory_df['Longitude'] >= bbox[0]) &
    (tree_inventory_df['Longitude'] <= bbox[1]) &
    (tree_inventory_df['Latitude'] >= bbox[2]) &
    (tree_inventory_df['Latitude'] <= bbox[3])
]


# Define a map and extent
m = Maps(crs=Maps.CRS.Mercator.GOOGLE, figsize=(20, 20))
m.set_extent((bbox[0], bbox[1], bbox[2], bbox[3]))

# Plot the GeoDataFrames onto the map
# Ground Truth Tree Inventory and Street View Image Locations
m.add_gdf(panos_gdf, marker='x', color='blue', alpha=0.80, markersize = 350, label='Panoramic Points')
m.add_gdf(tree_inventory_gdf, marker='^', color='orange', alpha=0.80, markersize = 350, label='Tree Inventory')
m.add_gdf(tree_est_gdf, marker='.', color=tree_est_gdf['color'], alpha=1.0, markersize=700, label='Estimated Tree Geolocation')

# Plot triangulared intersections
m.add_gdf(triangulated_tree_detections_gdf, marker='*', color='red', alpha=0.7, markersize = 400, label='Triangulated Intersections')
m.add_gdf(lines_gdf, color='black', alpha=0.7, label='Lines from Panoramic Images to Detected Trees')

m.add_wms.ESRI_ArcGIS.SERVICES.World_Imagery.add_layer.xyz_layer()

# Show and save the map
#m.savefig(f"C:/Users/talake2/Desktop/tree-geolocation/geolocation-ailanthus-testing-cities-results/{testing_city}/{testing_city}-initial-tree-geolocation-estimates-ailanthus-yolo.png")
m.show();


# Find Estimated Tree Locations Near Intersections

In [ ]:
# Create a KDTree to search for nearest neighbor search of intersection points by estimated tree locations
from scipy.spatial import KDTree

# Create a KD-Tree from the estimated tree locations to query against the intersections
# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KDTree.html
tree_points = tree_geolocation_results_df[['Est_Tree_Lon', 'Est_Tree_Lat']].values
tree_points_kdtree = KDTree(tree_points)


# Filter Intersections Less Than MaxDist from Estimated Tree Locations

In [ ]:

# Filter intersections based on distance ('maxdist') between estimated tree locations
filtered_intersections = []

# This dictionary will store the index of tree positions as keys and the 
# shortest distance to an assigned intersection and its point as values
assigned_positions = {}

# Define a maximum distance (meters) to search for intersections near two estimated tree geolocations
# If no intersections are found within (maxdist) of estimated tree locations, discard those intersections
#tree_geolocation_results_df['maxdist'] = 3 + (0.65 * tree_geolocation_results_df['Est_Depth'])
tree_geolocation_results_df['maxdist'] = 5

for point in range(len(triangulated_tree_detections_gdf.geometry)):
    intersection_point = triangulated_tree_detections_gdf.geometry[point]
    distances, indices = tree_points_kdtree.query((intersection_point.x, intersection_point.y), 2)
    distances_meters = distances * 111139  # Convert distances to meters
    
    # Loop through each of the closest tree position indices to this intersection
    for i, index in enumerate(indices):
        # Proceed only if the distance is within the max distance criteria.
        if distances_meters.max() <= tree_geolocation_results_df.loc[indices, 'maxdist'].max():
            # If this position is already assigned
            if index in assigned_positions:
                # Check if the current intersection is closer than the previously assigned one
                if distances_meters[i] < assigned_positions[index]['distance']:
                    # Update the stored distance and intersection point for this position
                    assigned_positions[index] = {'distance': distances_meters[i], 'intersection': intersection_point}
            else:
                # If it's not already assigned, assign this intersection to this tree position
                assigned_positions[index] = {'distance': distances_meters[i], 'intersection': intersection_point}

                
# After iterating through all intersections, add the assigned intersection points to filtered_intersections
for _, info in assigned_positions.items():
    # Add unique intersection points to the filtered list (checking existing points to avoid duplicates)
    if info['intersection'] not in filtered_intersections:
        filtered_intersections.append(info['intersection'])

df_intersections_filtered = pd.DataFrame([(point.x, point.y) for point in filtered_intersections], columns=['Lon', 'Lat'])

print(f'Filtering intersections to:', tree_geolocation_results_df['maxdist'][0], "meters from at least two estimated tree locations.")
print(f"Intersections before filtering:", len(triangulated_tree_detections_gdf.geometry))
print(f"Intersections after filtering:", len(filtered_intersections))


# Convert the list of Point objects to a GeoSeries
filtered_intersections_series = gpd.GeoSeries(filtered_intersections)

# Filter the triangulated_tree_detections_gdf to keep only the rows with 'Intersection' points present in filtered_intersections
filtered_tree_detections_gdf = triangulated_tree_detections_gdf[triangulated_tree_detections_gdf['geometry'].isin(filtered_intersections_series)]

# Convert to geodataframe
filtered_intersection_geometry = [Point(xy) for xy in filtered_tree_detections_gdf['Intersection']]
filtered_triangulated_tree_detections_gdf = gpd.GeoDataFrame(filtered_tree_detections_gdf, geometry=filtered_intersection_geometry, crs="EPSG:4326")

print(filtered_triangulated_tree_detections_gdf)

# Plot Triangulated Geolocation Estimates

In [ ]:
# Plot Pano Locations, Estimated Tree Locations, Lines and Intersections
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io import img_tiles
%matplotlib inline

# Define the bounding box for the plotting area
bbox = [
    tree_geolocation_results_df['Est_Tree_Lon'].min() - 0.0003,
    tree_geolocation_results_df['Est_Tree_Lon'].max() + 0.0003,
    tree_geolocation_results_df['Est_Tree_Lat'].min() - 0.0003,
    tree_geolocation_results_df['Est_Tree_Lat'].max() + 0.0003
]

# Crop the tree invenntory records to the bounding box where trees are predicted
cropped_tree_inventory = tree_inventory_df[
    (tree_inventory_df['Longitude'] >= bbox[0]) &
    (tree_inventory_df['Longitude'] <= bbox[1]) &
    (tree_inventory_df['Latitude'] >= bbox[2]) &
    (tree_inventory_df['Latitude'] <= bbox[3])
]


# Define a map and extent
m = Maps(crs=Maps.CRS.Mercator.GOOGLE, figsize=(20, 20))
m.set_extent((bbox[0], bbox[1], bbox[2], bbox[3]))

# Plot the GeoDataFrames onto the map
# Ground Truth Tree Inventory and Street View Image Locations
m.add_gdf(panos_gdf, marker='x', color='blue', alpha=0.80, markersize = 350, label='Panoramic Points')
m.add_gdf(tree_inventory_gdf, marker='^', color='orange', alpha=0.80, markersize = 350, label='Tree Inventory')
m.add_gdf(tree_est_gdf, marker='.', color=tree_est_gdf['color'], alpha=1.0, markersize=700, label='Estimated Tree Geolocation')

# Plot triangulared intersections
m.add_gdf(filtered_triangulated_tree_detections_gdf, marker='*', color='red', alpha=0.7, markersize = 400, label='Triangulated Intersections')
m.add_gdf(lines_gdf, color='black', alpha=0.7, label='Lines from Panoramic Images to Detected Trees')

m.add_wms.ESRI_ArcGIS.SERVICES.World_Imagery.add_layer.xyz_layer()

# Show and save the map
#m.savefig(f"C:/Users/talake2/Desktop/tree-geolocation/geolocation-ailanthus-testing-cities-results/{testing_city}/{testing_city}-initial-tree-geolocation-estimates-ailanthus-yolo.png")
m.show();


# Classify Tree Genus From Filtered Trianguled Detections

In [ ]:
pd.set_option('display.max_columns', None)
filtered_triangulated_tree_detections_gdf

In [ ]:
# Select only the columns ending with '_prob'
prob_columns = [col for col in filtered_triangulated_tree_detections_gdf.columns if col.endswith('_prob')]

# Plot bar plots for each of the selected columns
filtered_triangulated_tree_detections_gdf[prob_columns].plot(kind='bar', figsize=(12, 6))
plt.xlabel('Genus')  # Set x-axis label
plt.ylabel('Probability')  # Set y-axis label
plt.title('Probability of Each Genus')  # Set plot title
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.legend(loc='upper right')  # Show legend
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

# Apply Clustering to Group Adjacent Intersections based on Distance

In [ ]:
# Apply Hierarchical Clustering Based on Distance to Remaining Intersections/ Geolocated Tree Estimates
from sklearn.cluster import AgglomerativeClustering
# https://scikit-learn.org/stable/modules/clustering.html

final_tree_coordinates = []

# Convert the filtered intersection points to numpy array
intersection_points = np.array([(point.x, point.y) for point in filtered_intersections])

# Perform hierarchical clustering on the intersection points
# Distance threshold is in lat/long distances. 
# 10 meters/ 111139 = 9e-5
# 3 meters/ 111139 = 2.7e-5 Used by Lumnitz et al. 2021.
clustering = AgglomerativeClustering(distance_threshold=2.7e-5, n_clusters=None).fit(intersection_points)

# Assign cluster labels to each intersection point
cluster_labels = clustering.labels_

# Create a dictionary to store the cluster centers
cluster_centers = {}

# Organize the intersection points into clusters in the cluster_centers dictionary
for i, point in enumerate(intersection_points):
    if cluster_labels[i] not in cluster_centers:
        cluster_centers[cluster_labels[i]] = []
        cluster_centers[cluster_labels[i]].append((point[0], point[1]))
        
# Calculate the average location for each cluster and appends it to the final_tree_coordinates list
for cluster_label in cluster_centers:
    cluster_points = np.array(cluster_centers[cluster_label])
    average_location = np.mean(cluster_points, axis=0)
    final_tree_coordinates.append(average_location)

# # Convert the final tree coordinates back to DataFrame
df_final_tree_coordinates = pd.DataFrame(final_tree_coordinates, columns=['Lon', 'Lat'])

# Convert final geolocated trees to a Geodataframe
geolocated_trees_geometry = [Point(xy) for xy in zip(df_final_tree_coordinates['Lon'], df_final_tree_coordinates['Lat'])]
geolocated_trees_gdf = gpd.GeoDataFrame(df_final_tree_coordinates, geometry=geolocated_trees_geometry, crs="EPSG:4326")
geolocated_trees_gdf_reproj = geolocated_trees_gdf.to_crs(epsg=3395)


print(f"Applying Hierarchical Clustering to Geolocated Intersection Points")
print(f"Intersections before clustering:", len(filtered_intersections))
print(f"Intersections after clustering:", len(df_final_tree_coordinates))



In [ ]:
geolocated_trees_gdf_reproj

# Assign Predicted Genus to Final Geolocated Tree Coordinates

In [ ]:
geolocated_trees_gdf # Contains final geolocated trees

In [ ]:
tree_est_gdf # Contains initial geolocation estimates with predicted genus

In [ ]:

# Contains final geolocated tree coordinates
# Reproject to meters
geolocated_trees_gdf_reproj = geolocated_trees_gdf.to_crs(epsg=3395) # Reproject to meters

# Create a meter buffer around the tree_inventory_gdf points
buffer_distance = 5  # in meters
geolocated_trees_gdf_reproj_buffer = geolocated_trees_gdf_reproj.geometry.buffer(buffer_distance)

# Contains initial geolocation estimates with predicted genus
# Reproject to meters
tree_est_gdf_reproj = tree_est_gdf.to_crs(epsg=3395)# Reproject to meters

# Iterate over each buffer around geolocated trees
for idx, buffer_tree in enumerate(geolocated_trees_gdf_reproj_buffer):
    # Get initial geolocated tree estimates within buffer around final geolocated tree
    tree_est_in_geolocated = tree_est_gdf_reproj[tree_est_gdf_reproj.geometry.within(buffer_tree)]
    # Count the number of predicted genera near each final geolocated tree, and get most frequently predicted genus
    top_predicted_genera = tree_est_in_geolocated['Predicted_Genus'].value_counts().idxmax()
    # Assign the predicted genus to the final geolocated tree geodataframe
    geolocated_trees_gdf.loc[idx, 'Assigned_Genus'] = top_predicted_genera
    

    

# Plot Pano Locations, Estimated Tree Locations, Final Geolocated Tree Estimates

In [ ]:
# Plot Pano Locations, Estimated Tree Locations, Lines and Intersections
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io import img_tiles
%matplotlib inline

# Define the bounding box for the plotting area
bbox = [
    tree_geolocation_results_df['Est_Tree_Lon'].min() - 0.0003,
    tree_geolocation_results_df['Est_Tree_Lon'].max() + 0.0003,
    tree_geolocation_results_df['Est_Tree_Lat'].min() - 0.0003,
    tree_geolocation_results_df['Est_Tree_Lat'].max() + 0.0003
]

# Crop the tree invenntory records to the bounding box where trees are predicted
cropped_tree_inventory = tree_inventory_df[
    (tree_inventory_df['Longitude'] >= bbox[0]) &
    (tree_inventory_df['Longitude'] <= bbox[1]) &
    (tree_inventory_df['Latitude'] >= bbox[2]) &
    (tree_inventory_df['Latitude'] <= bbox[3])
]


# Define a map and extent
m = Maps(crs=Maps.CRS.Mercator.GOOGLE, figsize=(20, 20))
m.set_extent((bbox[0], bbox[1], bbox[2], bbox[3]))


# Define unique colors for each predicted genus
unique_genera = geolocated_trees_gdf['Assigned_Genus'].unique()
num_colors = len(unique_genera)
color_map = plt.cm.get_cmap('tab10', num_colors)  # You can choose any colormap here

# Create a dictionary mapping predicted genera to colors
genus_color_dict = {genus: color_map(i) for i, genus in enumerate(unique_genera)}

# Apply colors based on predicted genus
geolocated_trees_gdf['color'] = geolocated_trees_gdf['Assigned_Genus'].map(genus_color_dict)

# Plot the GeoDataFrame onto the map
m.add_gdf(panos_gdf, marker='x', color='blue', alpha=0.75, markersize = 300, label='Panoramic Points')
m.add_gdf(tree_inventory_gdf, marker='^', color='orange', alpha=0.75, markersize = 300, label='Tree Inventory')
m.add_gdf(tree_est_gdf, marker='.', color=tree_est_gdf['color'], alpha=0.75, markersize=300, label='Estimated Tree Geolocation')
m.add_gdf(geolocated_trees_gdf, marker='o', color=geolocated_trees_gdf['color'], alpha=0.75, markersize = 500, label='Triangulated Intersections')
m.add_wms.ESRI_ArcGIS.SERVICES.World_Imagery.add_layer.xyz_layer()

# Show the map
#m.savefig(f'C:/Users/talake2/Desktop/tree-geolocation/geolocation-panos-testing-cities-results/{testing_city}-tree-geolocation-results-pre-matching.png')
m.show();


# Tree Geolocation Model Evaluation by Matching

In [ ]:

def greedy_matching(tree_inventory_gdf, geolocated_trees_gdf, max_distance=15):
    """
    Greedy algorithm to match closest trees and calculate RMSE.
    Args:
        tree_inventory_gdf (gpd.GeoDataFrame): ground truth tree locations.
        geolocated_trees_gdf (gpd.GeoDataFrame): predicted tree locations.
        max_distance (float): maximum distance to consider a match as valid (in meters).
    Returns:
        (list of tuple): List of matched pairs (ground_truth_index, prediction_index).
        (gpd.GeoDataFrame): Dataframe of true positive matches with their distances.
        (float): RMSE of the matched pairs.
    """
    matches = []
    distances = []
    squared_distances = []
    for idx1, tree1 in tree_inventory_gdf.iterrows():
        closest_tree = None
        closest_dist = float('inf')
        for idx2, tree2 in geolocated_trees_gdf.iterrows():
            dist = calculate_distance(tree1.geometry, tree2.geometry)
            if dist < closest_dist:
                closest_dist = dist
                closest_tree = idx2
        if closest_tree is not None and closest_dist <= max_distance:
            matches.append((idx1, closest_tree))
            distances.append(closest_dist)
            squared_distances.append(closest_dist ** 2)  # Square the distance for RMSE calculation
            geolocated_trees_gdf = geolocated_trees_gdf.drop(closest_tree)
            
    true_positive_matches = tree_inventory_gdf.loc[[idx1 for idx1, idx2 in matches]]
    true_positive_matches['distance'] = [dist ** 0.5 for dist in squared_distances]  # Store sqrt(distance) which is the actual distance
    rmse = np.sqrt(np.mean(squared_distances)) if squared_distances else None
    mae = np.mean(distances) if distances else None
    return matches, true_positive_matches, rmse, mae

# Assuming tree_inventory_gdf and geolocated_trees_gdf are your ground truth and prediction GeoDataFrames respectively
matches, true_positives_gdf, rmse, mae = greedy_matching(tree_inventory_gdf, geolocated_trees_gdf, max_distance=15)

print(f"Number of true positive matches: ", len(matches), "out of total inventory trees: ", len(tree_inventory_gdf))
print(f"Root Mean Square Error (RMSE) between geolocated trees and inventory trees : {rmse:.2f} meters")
print(f"Mean Absolute Error (MAE) between geolocated trees and inventory trees : {mae:.2f} meters")

# Get Geolocated Trees that Match with Ground Truth 
matched_predictions_indices = [pred for _, pred in matches]
matched_predictions_gdf = geolocated_trees_gdf.loc[matched_predictions_indices]

In [ ]:

# Create histogram
plt.hist(list(true_positives_gdf['distance']), bins=100, range=(0, 30), color='blue', edgecolor='black');
plt.title(f'Distance From Matched Geolocated Tree to Ground Truth in {testing_city}');
plt.xlabel('Distance (meters)');
plt.ylabel('Frequency');
#plt.savefig(f'C:/Users/talake2/Desktop/tree-geolocation/geolocation-panos-evaluation/geolocation-panos-testing-cities-tree-detection-results/{testing_city}_matched_geolocated_trees_distance_ground_truth.png')
plt.show();


# Plot final geolocation estimates

In [ ]:
# Plot Pano Locations, Estimated Tree Locations, Lines and Intersections
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io import img_tiles
%matplotlib inline

# Define the bounding box for the plotting area
bbox = [
    tree_geolocation_results_df['Est_Tree_Lon'].min() - 0.0003,
    tree_geolocation_results_df['Est_Tree_Lon'].max() + 0.0003,
    tree_geolocation_results_df['Est_Tree_Lat'].min() - 0.0003,
    tree_geolocation_results_df['Est_Tree_Lat'].max() + 0.0003
]

# Crop the tree invenntory records to the bounding box where trees are predicted
cropped_tree_inventory = tree_inventory_df[
    (tree_inventory_df['Longitude'] >= bbox[0]) &
    (tree_inventory_df['Longitude'] <= bbox[1]) &
    (tree_inventory_df['Latitude'] >= bbox[2]) &
    (tree_inventory_df['Latitude'] <= bbox[3])
]


# Convert panos_df to a GeoDataFrame
geometry = [Point(xy) for xy in zip(tree_geolocation_results_df['Pano_Origin_Lon'],  tree_geolocation_results_df['Pano_Origin_Lat'])]
panos_gdf = gpd.GeoDataFrame(tree_geolocation_results_df, geometry=geometry, crs="EPSG:4326")

# Convert cropped_tree_inventory to a Geodataframe
tree_geometry = [Point(xy) for xy in zip(cropped_tree_inventory['Longitude'], cropped_tree_inventory['Latitude'])]
tree_inventory_gdf = gpd.GeoDataFrame(cropped_tree_inventory, geometry=tree_geometry, crs="EPSG:4326")

# Convert estimated tree geolocations to a Geodataframe
tree_est_geometry = [Point(xy) for xy in zip(tree_geolocation_results_df['Est_Tree_Lon'], tree_geolocation_results_df['Est_Tree_Lat'])]
tree_est_gdf = gpd.GeoDataFrame(tree_geolocation_results_df, geometry=tree_est_geometry, crs="EPSG:4326")

# Convert geolocated trees to a Geodataframe
geolocated_trees_geometry = [Point(xy) for xy in zip(df_final_tree_coordinates['Lon'], df_final_tree_coordinates['Lat'])]
geolocated_trees_gdf = gpd.GeoDataFrame(df_final_tree_coordinates, geometry=geolocated_trees_geometry, crs="EPSG:4326")


# Define a map and extent
m = Maps(crs=Maps.CRS.Mercator.GOOGLE, figsize=(20, 20))
m.set_extent((bbox[0], bbox[1], bbox[2], bbox[3]))

# Plot the GeoDataFrame onto the map
m.add_gdf(panos_gdf, marker='x', color='blue', alpha=0.75, markersize = 300, label='Panoramic Points')
m.add_gdf(tree_inventory_gdf, marker='^', color='orange', alpha=0.75, markersize = 300, label='Tree Inventory')
m.add_gdf(matched_predictions_gdf, marker='.', color='purple', alpha=0.75, markersize = 300, label='Matched Geolocated Trees')
m.add_wms.ESRI_ArcGIS.SERVICES.World_Imagery.add_layer.xyz_layer()

# Show the map
#m.savefig(f"C:/Users/talake2/Desktop/tree-geolocation/geolocation-panos-evaluation/geolocation-panos-testing-cities-tree-detection-results/{testing_city}-tree-geolocation-results-final.png")
m.show();


# Exports as .CSV

In [ ]:
# Create and format geolocation summary statistics for export

# Define the values for the geolocation summary statistics
city = testing_city
num_inventory_trees = len(cropped_tree_inventory)
num_matched_trees = len(matched_predictions_gdf)
freq_matched_trees = num_matched_trees / num_inventory_trees
min_geo_distance = np.min(list(true_positives_gdf['distance']))
max_geo_distance = np.max(list(true_positives_gdf['distance']))
mean_geo_distance = np.mean(list(true_positives_gdf['distance']))
median_geo_distance = np.median(list(true_positives_gdf['distance']))
avg_distance_pano_to_tree = np.mean(closest_distances)

# Create a dictionary with the summary statistics
summary_stats = {
    "Region": [city],
    "Number of inventory trees": [num_inventory_trees],
    "Number of matched trees": [num_matched_trees],
    "Error distance (RMSE)": [rmse],
    "Frequency of matched trees": [freq_matched_trees],
    "Minimum geolocation distance": [min_geo_distance],
    "Maximum geolocation distance": [max_geo_distance],
    "Mean geolocation distance": [mean_geo_distance],
    "Median geolocation distance": [median_geo_distance],
    "Average distance from pano to tree": [avg_distance_pano_to_tree]
}

# Create a DataFrame from the dictionary
summary_df = pd.DataFrame(summary_stats)

# Write the DataFrame to a CSV file
summary_df.to_csv(f'C:/Users/talake2/Desktop/tree-geolocation/geolocation-panos-testing-cities-results/{testing_city}_geolocation_summary.csv', index=False)




# Calculate Data Metrics: Nearest Neighbor Distances (Pano-Pano, Tree-Tree, Pano-Tree)

In [ ]:
panos_df = pd.DataFrame(columns=['Pano_ID', 'Pano_Origin_Lon', 'Pano_Origin_Lat'])

# Iterate over each image in the directory.
for img_name in os.listdir(img_folder):
    if img_name.endswith('.jpg'):
        
        # Read Panoramic image and Metadata .JSON file, return image and metadata
        img, width, pano_id, pano_rotation_value, pano_lat, pano_lon = read_panoramic_image_and_metadata(img_folder, img_name)

        new_row = {
        'Pano_ID': pano_id,
        'Pano_Origin_Lon': pano_lon,
        'Pano_Origin_Lat': pano_lat,
        }
                        
        panos_df = pd.concat([panos_df, pd.DataFrame([new_row])], ignore_index=True)
        
        

# Convert panos_df to a GeoDataFrame
geometry = [Point(xy) for xy in zip(panos_df['Pano_Origin_Lon'], panos_df['Pano_Origin_Lat'])]
panos_gdf = gpd.GeoDataFrame(panos_df, geometry=geometry, crs="EPSG:4326")

# Convert cropped_tree_inventory to a Geodataframe
tree_geometry = [Point(xy) for xy in zip(cropped_tree_inventory['Longitude'], cropped_tree_inventory['Latitude'])]
tree_inventory_gdf = gpd.GeoDataFrame(cropped_tree_inventory, geometry=tree_geometry, crs="EPSG:4326")


# Reproject to CRS that measures distances in meters (World Mercator)
pano_gdf = panos_gdf.to_crs(epsg=3395)  # World Mercator

# Extract the x and y coordinates of each point
pano_coordinates = np.array(list(pano_gdf.geometry.apply(lambda geom: (geom.x, geom.y))))

# Create a cKDTree for efficient spatial querying
tree = cKDTree(pano_coordinates)

# Query the nearest neighbor for each point (excluding itself). k=2 returns the first and second nearest neighbors, but we exclude the first as it is the point itself.
distances, _ = tree.query(pano_coordinates, k=2)
# distances contains two columns, where the second column is the distance to the nearest neighbor since the first is the distance to itself (zero)

# Get all nearest-neighbor distances
nearest_neighbor_distances = distances[:, 1]

# Calculate the average distance among the nearest neighbors
average_pano_pano_distance = np.mean(distances[:, 1]) # We select the second column corresponding to nearest neighbors
min_pano_pano_distance = np.max(distances[:, 1])
max_pano_pano_distance = np.min(distances[:, 1])

print(f"Average distance of panoramic images to the nearest neighbor: {average_pano_pano_distance} meters")
print(f"Max distance of panoramic images to the nearest neighbor: {min_pano_pano_distance} meters")
print(f"Min distance of panoramic images to the nearest neighbor: {max_pano_pano_distance} meters")


In [ ]:

# Assuming panos_gdf is your GeoDataFrame with Point geometries
# Ensure it is in a projected CRS that measures distances in meters (World Mercator)
inventory_gdf = tree_inventory_gdf.to_crs(epsg=3395)  # World Mercator

# Extract the x and y coordinates of each point
inv_coordinates = np.array(list(inventory_gdf.geometry.apply(lambda geom: (geom.x, geom.y))))

# Create a cKDTree for efficient spatial querying
tree = cKDTree(inv_coordinates)

# Query the nearest neighbor for each point (excluding itself). k=2 returns the first and second nearest neighbors, but we exclude the first as it is the point itself.
distances, _ = tree.query(inv_coordinates, k=2)
# distances contains two columns, where the second column is the distance to the nearest neighbor since the first is the distance to itself (zero)

# Get all nearest-neighbor distances
nearest_neighbor_distances = distances[:, 1]

# Calculate the average distance among the nearest neighbors
average_tree_tree_distance = np.mean(distances[:, 1]) # We select the second column corresponding to nearest neighbors
min_tree_tree_distance = np.max(distances[:, 1])
max_tree_tree_distance = np.min(distances[:, 1])

print(f"Average distance of tree inventory to the nearest neighbor: {average_tree_tree_distance} meters")
print(f"Max distance of tree inventory to the nearest neighbor: {min_tree_tree_distance} meters")
print(f"Min distance of tree inventory to the nearest neighbor: {max_tree_tree_distance} meters")


In [ ]:
distances

In [ ]:
from scipy import stats

# Build a spatial index (cKDTree) for 'inventory_gdf' coordinates
inventory_tree = cKDTree(inv_coordinates)

# For each point in 'panos_gdf', find the distance to the nearest point in 'inventory_gdf'
distances, _ = inventory_tree.query(pano_coordinates, k=1)

# Calculate the average distance between 'panos_gdf' points and nearest 'inventory_gdf' points
average_pano_tree_distance = np.mean(distances)
median_pano_tree_distance = np.median(distances)
min_pano_tree_distance = np.min(distances)
max_pano_tree_distance = np.max(distances)


print(f"Average nearest neighbor distance from panoramic images to inventory items: {average_pano_tree_distance} meters")
print(f"Median nearest neighbor distance from panoramic images to inventory items: {median_pano_tree_distance} meters")
print(f"Min nearest neighbor distance from panoramic images to inventory items: {min_pano_tree_distance} meters")
print(f"Max nearest neighbor distance from panoramic images to inventory items: {max_pano_tree_distance} meters")



# Inspect Panoramic Imagery Dimensions for Potential Issues

In [ ]:
# Inspect Panoramic Imagery Dimensions for Potential Issues

import os
import json

panos_dir = r"C:/Users/talake2/Desktop/tree-geolocation/geolocation-pano-testing-cities"

# Dictionary to store counts of images for each size
image_counts = {}

# For every directory in panos_dir
for folder in os.listdir(panos_dir):
    folder_path = os.path.join(panos_dir, folder)
    if os.path.isdir(folder_path):
        print(f"Folder Name: {folder}")
        image_counts[folder] = {}
        json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
        
        # Iterate through all .json files
        for json_file in json_files:
            json_path = os.path.join(folder_path, json_file)
            with open(json_path, 'r') as f:
                metadata = json.load(f)
                resolution = metadata.get('resolution', {})
                width = resolution.get('width', None)
                height = resolution.get('height', None)
                dimensions = (width, height)
                pano_id = metadata.get('panoId', None)
                date = metadata.get('date', None)
                   
                # Check if width is not 16384
                if width != 16384:
                    print(f"PanoID: {pano_id}, Width: {width}, Height: {height}")
                    print(f"Date: {date}")
                
                # Count number of .json files for each size
                if dimensions not in image_counts[folder]:
                    image_counts[folder][dimensions] = 1
                else:
                    image_counts[folder][dimensions] += 1

# Summarize folder name, number of files in each folder, and number of files of each size (width, height)
for folder, counts in image_counts.items():
    print(f"\nFolder Name: {folder}")
    print("Number of files in folder:", sum(counts.values()))
    for size, count in counts.items():
        print(f"File Size: {size}, Count: {count}")




In [ ]:
# EOF